## 📌 **Formularios POST en Django**  

El objetivo de este tema es que entiendas cómo enviar datos desde un formulario en el frontend hacia el backend de Django, procesarlos y almacenarlos en la base de datos.  

---

### 🔹 **1. ¿Qué es un formulario y cómo interactúa con Django?**  

Un **formulario** en una aplicación web es una interfaz que permite a los usuarios enviar datos al servidor. Esto se puede hacer con los métodos HTTP:  

- **GET** → Se usa para obtener datos (ejemplo: búsqueda en Google).  
- **POST** → Se usa para enviar datos sensibles (ejemplo: registro de usuarios).  

En Django, los formularios pueden manejarse de tres formas:  
1. **Formularios HTML básicos** (con `request.POST`).  
2. **Formularios con `forms.Form` o `forms.ModelForm`** (recomendado para manejar validaciones).  
3. **Formularios en APIs con Django REST Framework (DRF)** (cuando queremos consumirlos desde un frontend o una app móvil).  

Nos centraremos en la forma tradicional con `forms.Form` y `forms.ModelForm`, porque nos permiten validar y estructurar datos de forma eficiente.  

---

### 🔹 **2. Creando un formulario POST en Django (Ejemplo práctico)**  

📍 **Paso 1: Crear un modelo en `models.py`**  

Vamos a suponer que queremos permitir que los usuarios envíen un comentario en un blog:  

```python
from django.db import models

class Comentario(models.Model):
    nombre = models.CharField(max_length=100)
    email = models.EmailField()
    mensaje = models.TextField()
    fecha = models.DateTimeField(auto_now_add=True)

    def __str__(self):
        return f"{self.nombre} - {self.mensaje[:20]}..."
```

Aquí creamos un modelo llamado `Comentario` que almacenará los datos que el usuario envíe mediante el formulario.

---

📍 **Paso 2: Crear un formulario llamado `forms.py` en la misma carpeta donde esta `views.py`**  

En lugar de manejar el formulario manualmente con `request.POST`, Django nos permite usar `forms.ModelForm`, que facilita la validación y estructura de los datos:

```python
from django import forms
from .models import Comentario

class ComentarioForm(forms.ModelForm):
    class Meta:
        model = Comentario
        fields = ['nombre', 'email', 'mensaje']
```

¿Qué hace `forms.ModelForm`?  
✔️ Genera los campos del formulario automáticamente según el modelo.  
✔️ Aplica validaciones de forma automática.  
✔️ Nos permite trabajar con datos limpios (`form.cleaned_data`).  

---

📍 **Paso 3: Crear la vista en `views.py`**  

Ahora debemos procesar los datos del formulario en la vista:  

```python
from django.shortcuts import render, redirect
from .forms import ComentarioForm

def enviar_comentario(request):
    if request.method == 'POST':
        form = ComentarioForm(request.POST)
        if form.is_valid():
            form.save()  # Guarda el comentario en la base de datos
            return redirect('gracias')  # Redirige a una página de agradecimiento
    else:
        form = ComentarioForm()
    
    return render(request, 'comentario_form.html', {'form': form})
```

📌 **Explicación del flujo en la vista:**  
1. Si la petición es `POST`, Django toma los datos del formulario (`request.POST`).  
2. Se valida con `form.is_valid()`. Si es correcto, se guarda en la base de datos.  
3. Se redirige al usuario a una página de confirmación.  
4. Si la petición no es `POST`, solo se muestra el formulario vacío.  

---

📍 **Paso 4: Crear la plantilla en `templates/comentario_form.html`** (no templates/mi_app) 

```html
<form method="POST">
    {% csrf_token %}
    {{ form.as_p }} 
    <button type="submit">Enviar</button>
</form>
```

✔️ `{% csrf_token %}` protege contra ataques CSRF.  
✔️ `{{ form.as_p }}` muestra los campos del formulario con etiquetas `<p>`.  
✔️ Se usa `method="POST"` para enviar los datos al backend.  

---

📍 **Paso 5: Configurar la URL en `urls.py`**  

```python
from django.urls import path
from .views import enviar_comentario

urlpatterns = [
    path('comentario/', enviar_comentario, name='comentario'),
]
```

Ahora podemos acceder al formulario en `http://localhost:8000/comentario/` 🚀.  


### Genera y aplica las migraciones nuevamente
Ejecuta los siguientes comandos en la terminal dentro del entorno virtual:

```sh
python manage.py makemigrations mi_app
```

Si todo está bien, debería mostrar un mensaje como:

```
Migrations for 'mi_app':
  mi_app/migrations/0001_initial.py
    - Create model Comentario
```

Luego, aplica las migraciones a la base de datos:

```sh
python manage.py migrate
```

Si la migración es exitosa, deberías ver un mensaje como:

```
Applying mi_app.0001_initial... OK
```

#### 1️⃣ Redirigir a un `path` con el nombre `'gracias'`

Abre tu archivo **`urls.py`** y asegúrate de que tienes una ruta con el nombre `'gracias'`:

```python
from django.urls import path
from mi_app import views

urlpatterns = [
    path('gracias/', views.gracias, name='gracias'),  # Asegúrate de que esta línea existe
]
```

#### Crea una vista `gracias` en `views.py`
Ahora, abre `views.py` y asegúrate de crear una función llamada `gracias`:

```python
def gracias(request):
    return HttpResponse('Gracias por tu comentario!')
```

---

📌 **Resumen del flujo de trabajo:**  

1️⃣ El usuario llena el formulario en `comentario_form.html`.  
2️⃣ Los datos se envían vía `POST` a la vista `enviar_comentario()`.  
3️⃣ Django valida los datos con `forms.ModelForm`.  
4️⃣ Si es válido, se guarda en la base de datos y redirige al usuario.  
5️⃣ Si hay errores, se muestran nuevamente en el formulario.  

---

### ✅ **Bonus: Agregando validaciones personalizadas**  

Django ya tiene validaciones predeterminadas, pero podemos personalizarlas. Por ejemplo, validemos que el mensaje tenga al menos 10 caracteres:  

```python
class ComentarioForm(forms.ModelForm):
    class Meta:
        model = Comentario
        fields = ['nombre', 'email', 'mensaje']

    def clean_mensaje(self):
        mensaje = self.cleaned_data.get('mensaje')
        if len(mensaje) < 10:
            raise forms.ValidationError("El mensaje debe tener al menos 10 caracteres")
        return mensaje
```

Si el usuario escribe un mensaje corto, Django mostrará un error automáticamente en el formulario.  